# Group 17: 
### Eyad Medhat 221100279 / Hady Aly 221101190 / Mohamed Mahfouz 221101743 / Omar Mady 221100745

# 1- meta_data_preprocessing

In [1]:
import pandas as pd
import json
import os
import csv
import numpy as np

In [2]:
# # --- CONFIGURATION ---
# # Raw string for Windows path
# FILE_PATH = r"C:\Users\hadye\Downloads\meta_Health_and_Household.jsonl\meta_Health_and_Household.jsonl"
# OUTPUT_FILE = "health_household_processed.csv"
# CHUNK_SIZE = 10000

# # Logic A: Medical Exclusion Keywords (Strict Filter)
# # We exclude items if ANY of these terms appear in their categories
# EXCLUDED_CATEGORIES = {
#     'medication', 'pills', 'vitamins', 'supplements', 'drugs', 'pharmacy', 
#     'medicine', 'prescription', 'otc', 'capsules', 'tablets'
# }

# # Logic B: Green Lift Keywords (Sustainability Signals)
# # We verify items as green if they contain ANY of these keywords in their text blob
# GREEN_KEYWORDS = [
#     'climate pledge friendly', 'compact by design', 'biodegradable', 'epa safer choice', 
#     'rechargeable', 'reusable', 'compostable', 'plastic free', 'zero waste', 
#     'organic', 'energy star', 'bpa free', 'no butane', 'sustainable', 'ecologo', 
#     'cradle to cradle', 'made in green', 'gots certified'
# ]

# def is_green_product(text_blob):
#     """Checks if any green keyword exists in the text blob."""
#     if not isinstance(text_blob, str):
#         return 0
#     for keyword in GREEN_KEYWORDS:
#         if keyword in text_blob:
#             return 1
#     return 0

# def extract_image_url(images_list):
#     """Extracts the best available image URL (hi_res > large > thumb)."""
#     if isinstance(images_list, list) and len(images_list) > 0:
#         first_image = images_list[0]
#         # Prioritize high resolution, then large, then thumb
#         if isinstance(first_image, dict):
#             return first_image.get('hi_res') or first_image.get('large') or first_image.get('thumb')
#     return None

# def flatten_details(details_dict):
#     """Flattens the details dictionary into a string for TF-IDF."""
#     if isinstance(details_dict, dict):
#         # Convert {"Scent": "Lavender", "Brand": "X"} -> "Scent Lavender Brand X"
#         return " ".join([f"{k} {v}" for k, v in details_dict.items() if isinstance(v, str)])
#     return ""

# def process_chunk(chunk):
#     """Clean and process a single chunk of data."""
#     # 1. Feature Extraction & Text Normalization
    
#     # Text Fields
#     chunk['features_str'] = chunk['features'].apply(lambda x: " ".join(x) if isinstance(x, list) else "")
#     chunk['description_str'] = chunk['description'].apply(lambda x: " ".join(x) if isinstance(x, list) else "")
#     chunk['details_str'] = chunk['details'].apply(flatten_details)
    
#     # Combined Text Blob for TF-IDF (Key for Content-Based Filtering)
#     chunk['full_text'] = (
#         chunk['title'].fillna('') + " " + 
#         chunk['features_str'] + " " + 
#         chunk['description_str'] + " " + 
#         chunk['details_str']
#     ).str.lower()
    
#     # New Features
#     chunk['image_url'] = chunk['images'].apply(extract_image_url)
#     chunk['price'] = chunk['price'].astype(str).replace('nan', '') # Keep original string format like "$19.99"
#     chunk['store'] = chunk['store'].fillna('')
    
#     # 2. Logic B: Green Lift
#     chunk['is_green'] = chunk['full_text'].apply(is_green_product)
    
#     # 3. Logic A: Medical Exclusion Filter
#     # Create a string representation of categories for searching
#     chunk['categories_str'] = chunk['categories'].apply(lambda x: " ".join(x).lower() if isinstance(x, list) else "")
    
#     # Identify excluded rows
#     def has_excluded_term(cat_str):
#         if not cat_str: return False
#         for term in EXCLUDED_CATEGORIES:
#             if term in cat_str:
#                 return True
#         return False

#     mask_exclude = chunk['categories_str'].apply(has_excluded_term)
    
#     # Filter: Keep non-excluded items
#     clean_chunk = chunk[~mask_exclude].copy()
    
#     # Return only required columns
#     cols_to_keep = ['parent_asin', 'title', 'average_rating', 'rating_number', 'is_green', 'price', 'store', 'image_url']
    
#     # Select columns if they exist to be safe
#     existing_cols = [c for c in cols_to_keep if c in clean_chunk.columns]
    
#     return clean_chunk[existing_cols]

# def main():
#     print(f"Starting processing of {FILE_PATH}...")
    
#     total_processed_rows = 0
#     total_green_items = 0
#     chunk_count = 0
    
#     # Remove existing output file if it exists to start fresh
#     if os.path.exists(OUTPUT_FILE):
#         try:
#             os.remove(OUTPUT_FILE)
#             print(f"Removed existing output file: {OUTPUT_FILE}")
#         except OSError as e:
#             print(f"Error removing file: {e}")
#             return

#     try:
#         # Stream processing with chunks
#         # lines=True is required for JSONL
#         with pd.read_json(FILE_PATH, lines=True, chunksize=CHUNK_SIZE) as reader:
#             for chunk in reader:
#                 chunk_count += 1
                
#                 # Process the chunk
#                 processed_chunk = process_chunk(chunk)
                
#                 # Update stats
#                 rows_in_chunk = len(processed_chunk)
#                 green_in_chunk = processed_chunk['is_green'].sum()
                
#                 total_processed_rows += rows_in_chunk
#                 total_green_items += green_in_chunk
                
#                 # Save to CSV (Append mode)
#                 # Write header only for the first chunk
#                 header_mode = True if chunk_count == 1 else False
#                 processed_chunk.to_csv(OUTPUT_FILE, mode='a', header=header_mode, index=False)
                
#                 print(f"Processed chunk {chunk_count}... (Kept {rows_in_chunk} items, {green_in_chunk} green)")
                
#     except ValueError as e:
#         print(f"Error parsing JSON: {e}")
#         print("Make sure the file is a valid JSONL file.")
#     except FileNotFoundError:
#         print(f"File not found: {FILE_PATH}")
#         print("Please check the path and try again.")
#     except Exception as e:
#         print(f"An unexpected error occurred: {e}")
#     else:
#         print("\n--- Processing Complete ---")
#         print(f"Total clean rows saved: {total_processed_rows}")
#         print(f"Total Green items found: {total_green_items}")
#         if total_processed_rows > 0:
#             print(f"Green Item Percentage: {(total_green_items/total_processed_rows)*100:.2f}%")
#         print(f"Saved to: {os.path.abspath(OUTPUT_FILE)}")

# if __name__ == "__main__":
#     main()
    
    


# 2- reviews_preprocessing

In [3]:
# # --- CONFIGURATION ---
# PRODUCTS_FILE = "health_household_processed.csv"
# REVIEWS_FILE = r"C:\Users\hadye\Downloads\meta_Health_and_Household.jsonl\Health_and_Household.jsonl"
# OUTPUT_FILE = "reviews_processed.csv"
# CHUNK_SIZE = 50000 

# def load_valid_asins(products_path):
#     """Loads a set of valid parent_asins from the processed products file."""
#     print(f"Loading valid products from {products_path}...")
#     try:
#         # We only need the parent_asin column
#         df = pd.read_csv(products_path, usecols=['parent_asin'])
#         valid_asins = set(df['parent_asin'].unique())
#         print(f"Loaded {len(valid_asins)} valid unique ASINs.")
#         return valid_asins
#     except FileNotFoundError:
#         print(f"Error: Products file not found at {products_path}")
#         return set()

# def main():
#     valid_asins = load_valid_asins(PRODUCTS_FILE)
    
#     if not valid_asins:
#         print("No valid ASINs found. Aborting.")
#         return

#     print(f"Starting processing of reviews from {REVIEWS_FILE}...")
    
#     # Remove existing output file
#     if os.path.exists(OUTPUT_FILE):
#         try:
#             os.remove(OUTPUT_FILE)
#             print(f"Removed existing output file: {OUTPUT_FILE}")
#         except OSError:
#             pass

#     total_reviews = 0
#     kept_reviews = 0
#     chunk_count = 0
    
#     try:
#         # Stream processing
#         with pd.read_json(REVIEWS_FILE, lines=True, chunksize=CHUNK_SIZE) as reader:
#             for chunk in reader:
#                 chunk_count += 1
                
#                 # Filter: Keep reviews where parent_asin is in our valid set
#                 # Note: Reviews file has 'parent_asin' or 'asin'? 
#                 # According to inspection, keys are ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'verified_purchase', 'helpful_vote']
#                 # We use 'parent_asin' to join.
                
#                 # Ensure parent_asin is loaded
#                 if 'parent_asin' not in chunk.columns:
#                      print("Warning: 'parent_asin' column missing in chunk.")
#                      continue
                
#                 mask = chunk['parent_asin'].isin(valid_asins)
#                 filtered_chunk = chunk[mask]
                
#                 rows_in_chunk = len(chunk)
#                 rows_kept = len(filtered_chunk)
                
#                 total_reviews += rows_in_chunk
#                 kept_reviews += rows_kept
                
#                 if rows_kept > 0:
#                     # Select useful columns
#                     cols = ['parent_asin', 'user_id', 'rating', 'title', 'text', 'timestamp', 'verified_purchase', 'helpful_vote']
#                     existing_cols = [c for c in cols if c in filtered_chunk.columns]
                    
#                     header_mode = True if (chunk_count == 1) or (kept_reviews == rows_kept) else False 
#                     # Actually, header logic with chunking/skipping is tricky. 
#                     # Better: write header only if file is new.
#                     write_header = not os.path.exists(OUTPUT_FILE)
                    
#                     filtered_chunk[existing_cols].to_csv(OUTPUT_FILE, mode='a', header=write_header, index=False)
                
#                 print(f"Processed chunk {chunk_count} (Rows: {rows_in_chunk}, Kept: {rows_kept})")
                
#     except Exception as e:
#         print(f"Error processing reviews: {e}")
    
#     print("\n--- Reviews Processing Complete ---")
#     print(f"Total reviews scanned: {total_reviews}")
#     print(f"Total reviews saved: {kept_reviews}")
#     if total_reviews > 0:
#         print(f"Retention Rate: {(kept_reviews/total_reviews)*100:.2f}%")

# if __name__ == "__main__":
#     main()


# 3- join_meta_and_reviews

In [4]:
# # --- CONFIGURATION ---
# PRODUCTS_FILE = "health_household_processed.csv"
# REVIEWS_FILE = "reviews_processed.csv"
# OUTPUT_FILE = "merged_dataset.csv"

# def main():
#     print("Starting dataset merge...")
    
#     # Check if files exist
#     if not os.path.exists(PRODUCTS_FILE) or not os.path.exists(REVIEWS_FILE):
#         print(f"Error: Missing input files.\nProducts: {os.path.exists(PRODUCTS_FILE)}\nReviews: {os.path.exists(REVIEWS_FILE)}")
#         return

#     try:
#         print(f"Loading {PRODUCTS_FILE}...")
#         products = pd.read_csv(PRODUCTS_FILE)
#         print(f"Loaded {len(products)} products.")
        
#         print(f"Loading {REVIEWS_FILE}...")
#         # Reviews might be large, but let's try loading it as we filtered it significantly
#         reviews = pd.read_csv(REVIEWS_FILE)
#         print(f"Loaded {len(reviews)} reviews.")
        
#         print("Merging datasets on 'parent_asin'...")
#         # Inner join to keep only reviews for products we have (and vice-versa, though reviews were already filtered)
#         merged_df = pd.merge(reviews, products, on='parent_asin', how='inner')
        
#         print(f"Merge complete. Resulting shape: {merged_df.shape}")
        
#         print(f"Saving to {OUTPUT_FILE}...")
#         merged_df.to_csv(OUTPUT_FILE, index=False)
#         print("Success! Merged dataset saved.")
        
#         # Display sample
#         print("\nFirst 5 rows:")
#         print(merged_df.head())
        
#     except Exception as e:
#         print(f"An error occurred during merging: {e}")

# if __name__ == "__main__":
#     main()



# 4- Amazon_health&household Preprocessing

In [5]:
# --- CONFIGURATION ---
INPUT_FILE = "../data/Amazon_health&household.csv"
OUTPUT_FILE = "../data/Amazon_health&household_sampled.csv"

N_ITEMS = 1000
N_USERS = 10000
N_RATINGS = 100000
SEED = 42

In [6]:
print(f"Starting Data Sampling...")

# Check if file exists relative to the notebook
if not os.path.exists(INPUT_FILE):
    print(f"ERROR: Input file not found at {os.path.abspath(INPUT_FILE)}")
else:
    print(f"Input File: {os.path.abspath(INPUT_FILE)}")
    
    print("Loading data...")
    try:
        df = pd.read_csv(INPUT_FILE, dtype={'price': str}, low_memory=False)
        print(f"Loaded Data Shape: {df.shape}")
    except Exception as e:
        print(f"Error loading data: {e}")

Starting Data Sampling...
Input File: e:\AIE425_FinalProject_Group17\SECTION2_DomainRecommender\data\Amazon_health&household.csv
Loading data...
Loaded Data Shape: (7751115, 15)


In [7]:
# Drop unused columns
columns_to_drop = ["timestamp", "image_url", "store", "rating_number", "helpful_vote", "verified_purchase", "parent_asin"]
df.drop(columns=[c for c in columns_to_drop if c in df.columns], axis=1, inplace=True)

# Clean up column names (strip whitespace)
df.columns = df.columns.str.strip()

# Ensure 'item' column exists
if 'title_y' in df.columns and 'item' not in df.columns:
    print("Renaming 'title_y' to 'item'...")
    df.rename(columns={'title_y': 'item'}, inplace=True)

if 'item' not in df.columns:
     print("Warning: 'item' column not found. Checking for 'parent_asin'...")
     if 'parent_asin' in df.columns:
         df['item'] = df['parent_asin']
     else:
         print("Error: Could not identify item column.")

# Preview the cleaned data
df.head()

Renaming 'title_y' to 'item'...


,user_id,rating,title_x,text,item,average_rating,is_green,price
0,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,5,Yarn storage. Excellent.,I use these to store yarn projects. I’m very ...,"Hefty Slider Jumbo Storage Bags, 2.5 Gallon Si...",4.8,1,4.39
1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,5,Smells great!,Smells great!,Febreze AIR Effects Air Freshener Heavy Duty C...,4.6,0,NaN
2,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,5,Excellent!,Where has this been all my life? Makes wrappi...,Glad Press'n Seal Plastic Food Wrap - 70 Squar...,4.7,1,3.32
3,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1,Yuck!,Amazon was out of my regular brand of Charmin ...,Cottonelle Ultra ComfortCare Soft Toilet Paper...,4.5,0,29.75
4,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,5,only brand I use,Like using a cloud on your nether regions. I ...,"Charmin Ultra Gentle Toilet Paper, 18 Mega Rol...",4.7,1,19.97


In [8]:
print(f"Selecting Top {N_ITEMS} Items...")

top_items = df['item'].value_counts().nlargest(N_ITEMS).index
df_filtered = df[df['item'].isin(top_items)]

print(f"Rows after Item Filter: {len(df_filtered)}")

Selecting Top 1000 Items...
Rows after Item Filter: 2045696


In [9]:
print(f"Selecting Random {N_USERS} Users (Seed {SEED})...")
np.random.seed(SEED)

available_users = df_filtered['user_id'].unique()

if len(available_users) > N_USERS:
    selected_users = np.random.choice(available_users, N_USERS, replace=False)
    df_filtered = df_filtered[df_filtered['user_id'].isin(selected_users)]
else:
    print(f"Note: Only {len(available_users)} users available (requested {N_USERS}). keeping all.")

print(f"Rows after User Filter: {len(df_filtered)}")

Selecting Random 10000 Users (Seed 42)...
Rows after User Filter: 14554


In [10]:
print(f"Sampling {N_RATINGS} Interactions (Seed {SEED})...")

if len(df_filtered) > N_RATINGS:
    df_filtered = df_filtered.sample(n=N_RATINGS, random_state=SEED)
else:
     print(f"Note: Only {len(df_filtered)} interactions available (requested {N_RATINGS}). keeping all.")
     
print(f"Current Shape: {df_filtered.shape}")

Sampling 100000 Interactions (Seed 42)...
Note: Only 14554 interactions available (requested 100000). keeping all.
Current Shape: (14554, 8)


In [11]:
print(f"Final Shape: {df_filtered.shape}")
print(f"Unique Users: {df_filtered['user_id'].nunique()}")
print(f"Unique Items: {df_filtered['item'].nunique()}")

try:
    df_filtered.to_csv(OUTPUT_FILE, index=False)
    print(f"Saved sampled dataset to: {os.path.abspath(OUTPUT_FILE)}")
except Exception as e:
    print(f"Error saving file: {e}")

Final Shape: (14554, 8)
Unique Users: 10000
Unique Items: 1000
Saved sampled dataset to: e:\AIE425_FinalProject_Group17\SECTION2_DomainRecommender\data\Amazon_health&household_sampled.csv


In [12]:
from sklearn.preprocessing import LabelEncoder

print("Applying Label Encoding...")

# 1. Initialize Encoder
le = LabelEncoder()

# 2. Fit and Transform the 'item' column
# This converts strings to numbers (0 to N_ITEMS-1)
df_filtered['item_id_encoded'] = le.fit_transform(df_filtered['item'])

# 3. Create a mapping dictionary (Optional, but good for reference)
# So you know that 42 = "Colgate Toothpaste"
item_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

print(f"Encoded {len(item_mapping)} unique items.")
print("Example Mapping:")
for k, v in list(item_mapping.items())[:5]:
    print(f"  {k} -> {v}")

# 4. Save the compact file
# We save the encoded ID, but we can also keep the original 'item' name if you want reference
OUTPUT_ENCODED_FILE = "../data/Amazon_health&household_label_encoded.csv"

# Reorder columns to be cleaner
cols = ['user_id', 'item', 'item_id_encoded', 'rating','price','text', 'is_green'] 
# Add any other columns you have (like 'timestamp') to this list if they exist

df_filtered[cols].to_csv(OUTPUT_ENCODED_FILE, index=False)

print(f"\nSaved compact dataset to: {os.path.abspath(OUTPUT_ENCODED_FILE)}")
print(f"File preview:")
df_filtered[cols].head()

Applying Label Encoding...
Encoded 1000 unique items.
Example Mapping:
  (20 Pack) Extra Large Eraser Sponge - Extra Thick, Long Lasting, Premium Melamine Sponges in Bulk - Multi Surface Power Scrubber Foam Cleaning Pads - Bathtub, Floor, Baseboard, Bathroom, Wall Cleaner -> 0
  1 BY ONE Scale for Body Weight, Smart Body Fat Scale, Digital Bathroom Weighing Scale with Water Percentage Muscle Mass Bluetooth BMI, 14 Body Composition Analyzer, 400 lb -> 1
  1 Pair Happystep Comfort Memory Foam Insoles, Orthotic PU Shoe Inserts, Arch Support, Heel Cushioning, Shock Absorption, Plantar Fasciitis Foot Pain Relief for Men and Women (US W: 5-6) -> 2
  1 X 10 Energizer 377 376 Watch Battery SR626SW SR626W -> 3
  100% Compostable 9 Inch Paper Plates [125-Pack] Heavy-Dut, Natural Disposable Bagasse Plate, Eco-Friendly Made of Sugarcane Fibers - Natural Unbleached Brown 9" Biodegradable Plate by Stack Man -> 4

Saved compact dataset to: e:\AIE425_FinalProject_Group17\SECTION2_DomainRecommender\dat

,user_id,item,item_id_encoded,rating,price,text,is_green
1279,AEV4DP5E3FJH6FHDLXUYQTDEQYCQ,Sonic Handheld Percussion Massage Gun - Deep T...,827,2,79.99,"This product worked great when it worked, but ...",1
2706,AEHWUHNTB5FX32HJ7UBOZ2WWUX3Q,DUDE Wipes On-The-Go Flushable Wet Wipes - 1 P...,255,5,6.48,These are amazing for travel or for keeping in...,1
2708,AEHWUHNTB5FX32HJ7UBOZ2WWUX3Q,"Sleep Mask for Side Sleeper, 100% Blackout 3D ...",814,5,12.69,These are great! My other sleep mask pressed o...,0
2709,AEHWUHNTB5FX32HJ7UBOZ2WWUX3Q,"Cottonelle Freshfeel Flushable Wet Wipes, Adul...",226,5,15.79,These are really good quality. Do not tear lik...,1
2710,AEHWUHNTB5FX32HJ7UBOZ2WWUX3Q,"Silk Sleep Eye Mask for Men Women, Comfortable...",808,5,8.88,Great for travel. Super soft and silky. Has ad...,0
